In [1]:
from selenium import webdriver
from selenium.webdriver.edge.options import Options
from selenium.webdriver.edge.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import time
import winsound
import calendar
import datetime
import time
import re
import os

In [2]:
name = "m_enginering1.txt"
with open(name, "r") as f:
    line = f.read().strip() 
    downloaded = int(line)
    cur_downloaded = 0

In [3]:
def startdownload(download_dir, url, name):
    global cur_downloaded, downloaded
    with open(name, "r") as f:
        line = f.read().strip() 
        downloaded = int(line)
        cur_downloaded = 0
    print(cur_downloaded, downloaded)
    # download_dir = r"D:\zhiwangpdf\0715"
    edge_options = Options()
    prefs = {
        "download.default_directory": download_dir,
        "profile.default_content_settings.popups": 0,
        "directory_upgrade": True
    }
    User_Agent='Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36'
    edge_options.add_argument(f"user-agent={User_Agent}")
    edge_options.add_experimental_option("prefs", prefs)
    # edge_options.add_argument("--headless")  # 在后台运行浏览器

    # 设置Edge驱动路径
    webdriver_service = Service(r"E:\bingdownload\edgedriver_win64\msedgedriver.exe")
    # url = "https://cdfd.cnki.net/dejournals/#/cdfd/science"
    driver = webdriver.Edge(service=webdriver_service, options=edge_options)
    driver.get(url)
    time.sleep(2)
    
    #查找开始下载的year_month
    month_elmets = driver.find_elements(By.XPATH, '//div[@class="conneritembottomr"]/ul/li/a')
    i, j  = 0, len(month_elmets)
    print(i, j)
    while i < j:
        month_elmets[i].click()
        time.sleep(5)
        try:
            nums = driver.find_element(By.XPATH, '//div[@class="contentsbottomrb"]/div/span').text
            nums_list = re.findall(r"\d+", nums)
            nums = int("".join(nums_list))
            print(f"year month has pdfs:{nums}")
        except Exception as e:
            print(f"year month has pdfs:{nums}")
            print(e)
            nums = 0
        if nums == 0:
            driver.quit()
            driver = None
            break
        i += 1
        if cur_downloaded + nums <= downloaded:
            cur_downloaded += nums
            driver.back()
            time.sleep(5)
        else:
            break
        month_elmets = driver.find_elements(By.XPATH, '//div[@class="conneritembottomr"]/ul/li/a')
    return driver

def download_onepage(driver, name):
    # print(button[1].get_attribute("disabled"))
    cnt = 0
    flag = download_pdfs(driver, name)
    cnt += flag
    button = driver.find_elements(By.XPATH, '//div[@class="contentsbottomrb"]//button')
    while button[1].get_attribute("disabled") == None:
        button[1].click()
        # time.sleep(4)
        flag = download_pdfs(driver, name)
        cnt += flag
        if cnt == 2:
            driver.quit()
            return None
        button = driver.find_elements(By.XPATH, '//div[@class="contentsbottomrb"]//button')
    # flag = download_pdfs(driver, name)
    driver.quit()
    return None

def download_pdfs(driver, name):
    global downloaded, cur_downloaded 
    pdfs = driver.find_elements(By.XPATH, "//tbody/tr[@class='el-table__row']/td[1]/div[@class='cell']/a")
    main_window = driver.current_window_handle
    if cur_downloaded + len(pdfs) <= downloaded:
        cur_downloaded += len(pdfs)
        time.sleep(1)
        return False
    else:
        time.sleep(4)
        pdfs = driver.find_elements(By.XPATH, "//tbody/tr[@class='el-table__row']/td[1]/div[@class='cell']/a")
    i = 1
    window_set = set([main_window])
    for pdf in pdfs:
        try:
            print(pdf.text)
            pdf.click()
            time.sleep(2)
        except Exception as e:
            print("pdf html error.")
        windows = driver.window_handles
        # 切换到新窗口
        for window in windows:
            if window not in window_set:
                driver.switch_to.window(window)
                window_set.add(window)
                break
        try:
            dl_pdf = driver.find_element(By.XPATH, "//div[@id='DownLoadParts']/div/ul/li[3]/a")
            dl_pdf.click()
            time.sleep(2)
            #添加新产生的页面
            windows = driver.window_handles
            for window in windows:
                if window not in window_set:
                    window_set.add(window)
        except Exception as e:
            print(f"An error occurred")
        # time.sleep(2)
        driver.switch_to.window(main_window)
        i += 1
    cur_downloaded += len(pdfs)
    windows = driver.window_handles
    # 切换到新窗口
    time.sleep(20)
    for window in windows:
        try:
            if window != main_window:
                driver.switch_to.window(window)
                driver.close()
                time.sleep(0.5)
        except Exception as e:
            print(f"An error occurred when close window")
    driver.switch_to.window(main_window)
    with open(name, "w") as f:
        f.write(f"{cur_downloaded}")
    return True

In [4]:
while True:
    url = "https://cdfd.cnki.net/dejournals/#/cmfd/engineering1"
    download_dir = r"G:\zhiwangpdf\0716"
    try:
        driver = startdownload(download_dir, url, name)
        download_onepage(driver, name)
    except Exception as e:
        print(e)

0 1660
0 243
year month has pdfs:8758
PEMFC阴极催化层结构组分及其降解对反应传输影响的LBM研究
石墨烯增强银基电接触材料的制备与性能
燃料电池混合动力汽车集成变换器及其控制策略研究
基于改进雨流计数法的燃料电池性能衰减预测研究
MOFs衍生的钨基电催化剂在锌-空气电池中的应用
金属-氮掺杂的碳基材料制备及电催化性能研究
An error occurred
工况下燃料电池性能衰减建模及寿命预测
非铂催化剂为载体的燃料电池低铂催化剂研究
PEMFC中双极板和气体扩散层表面防护涂层的制备及性能研究
含多联苯结构的阴离子交换膜的制备与性能研究
连接体涂层用尖晶石型高熵氧化物的制备及界面反应研究
钴基金属有机框架材料衍生物的制备及电催化性能研究
Ga、Zn及Mn元素协同调控2N纯度铝制备高性能铝空气电池阳极材料的研究
Sr掺杂双钙钛矿材料La0.5Gd0.5Ba1-xSrxCo2O5+δ的制备与性能研究
钇稳定氧化锆固体氧化物电解质的制备及电学性能研究
集成储能装置的生物质燃料电池联合系统设计与性能研究
微流体微生物燃料电池性能分析与抗振性研究
流场结构及运行变量对PEM燃料电池性能影响的实验验证与模拟研究
An error occurred
基于深度学习的燃料电池故障诊断及剩余寿命预测的研究
电子结构调制对FeNx-Gra氧还原电催化剂性能影响的理论研究
An error occurred when close window
电化学法处理结晶紫和有机除草剂的初探
以肼为燃料、石墨为电极的棉线基微流体燃料电池性能初探
燃料电池机车液氢储存系统泄漏危害研究
固体氧化物燃料电池金属双极板（Ti,Nb）3SiC2涂层的制备及其高温性能研究
金刚石合成块自动化研磨生产线关键技术研究
单晶金刚石磨粒超精密加工织构表面碳化硅的机理研究
水润滑下超精密加工单晶立方碳化硅的原子模拟研究
聚晶金刚石电火花加工工艺及机理研究
B-S共掺杂单晶金刚石的制备与性能研究
纳米碳化硅及其复合材料的制备与降解性能研究
同轴数字压缩全息石英玻璃内部缺陷测量技术
高炉渣基微晶玻璃制备及性能研究
用于X-Ray探测的硼硅酸盐闪烁玻璃的设计制备及性能研究
CsPbBr3纳米晶玻璃制备及其可控析晶的研究
无容器技术在含钛氧化物玻璃的制备及热物性测试的应用研究
基于透射特性